# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234082502138                   -0.50    7.0         
  2   -7.250088899267       -1.80       -1.39    1.0   7.54ms
  3   -7.251243211937       -2.94       -2.06    2.0   9.29ms
  4   -7.251135417058   +   -3.97       -2.00    2.0   9.47ms
  5   -7.251330469451       -3.71       -2.71    1.0   7.74ms
  6   -7.251338084803       -5.12       -3.14    1.0   7.84ms
  7   -7.251338758530       -6.17       -3.86    1.0   7.80ms
  8   -7.251338796908       -7.42       -4.29    2.0   9.60ms
  9   -7.251338798327       -8.85       -4.90    1.0   7.98ms
 10   -7.251338798682       -9.45       -5.33    2.0   10.0ms
 11   -7.251338798702      -10.68       -5.84    1.0   8.18ms
 12   -7.251338798704      -11.92       -6.05    2.0   9.62ms
 13   -7.251338798705      -12.04       -6.85    1.0   8.19ms
 14   -7.251338798705      -13.48       -7.41    2.0   9.62ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0764022566735025
[ Info: Arnoldi iteration step 2: normres = 0.47338454155598697
[ Info: Arnoldi iteration step 3: normres = 0.833142785392378
[ Info: Arnoldi iteration step 4: normres = 0.23089473817528008
[ Info: Arnoldi iteration step 5: normres = 0.42559030520338154
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.89e-02, 3.62e-02, 3.43e-01, 2.48e-01, 1.58e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3275798419226944
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.79e-03, 2.78e-01, 1.22e-01, 9.77e-02, 5.99e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06978768585396153
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.97e-04, 1.62e-02, 4.51e-03, 2.73e-02, 5.23e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1090464803824848
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.02e-0